### Distance sensor

Formula: 34300cm = Distance / (pulse_duration / 2) -> Distance = 17150 * pulse_duration

Links:
+ https://thepihut.com/blogs/raspberry-pi-tutorials/hc-sr04-ultrasonic-range-sensor-on-the-raspberry-pi

In [ ]:
import RPi.GPIO as GPIO
from time import sleep, time

TRIG = 23
ECHO = 24

def get_distance():
    GPIO.output(TRIG, False)
    sleep(1)

    GPIO.output(TRIG, True)
    sleep(0.00001)
    GPIO.output(TRIG, False)

    pulse_start = time()
    while GPIO.input(ECHO) == 0:
        pulse_start = time()

    pulse_end = time()
    while GPIO.input(ECHO) == 1:
        pulse_end = time()

    pulse_duration = pulse_end - pulse_start
    return 17150 * pulse_duration

### LCD

In [ ]:
from smbus import SMBus

ALIGN_FUNC = {
    'left': 'ljust',
    'right': 'rjust',
    'center': 'center'}
CLEAR_DISPLAY = 0x01
ENABLE_BIT = 0b00000100
LINES = {
    1: 0x80,
    2: 0xC0,
    3: 0x94,
    4: 0xD4}

LCD_BACKLIGHT = 0x08
LCD_NOBACKLIGHT = 0x00

class LCD:

    def __init__(self, address=0x27, bus=1, width=16, rows=2, backlight=True):
        self.address = address
        self.bus = SMBus(bus)
        self.delay = 0.0005
        self.rows = rows
        self.width = width
        self.backlight_status = backlight

        self.write(0x33)
        self.write(0x32)
        self.write(0x06)
        self.write(0x0C)
        self.write(0x28)
        self.write(CLEAR_DISPLAY)
        sleep(self.delay)

    def _write_byte(self, byte):
        self.bus.write_byte(self.address, byte)
        self.bus.write_byte(self.address, (byte | ENABLE_BIT))
        sleep(self.delay)
        self.bus.write_byte(self.address,(byte & ~ENABLE_BIT))
        sleep(self.delay)

    def write(self, byte, mode=0):
        backlight_mode = LCD_BACKLIGHT if self.backlight_status else LCD_NOBACKLIGHT
        self._write_byte(mode | (byte & 0xF0) | backlight_mode)
        self._write_byte(mode | ((byte << 4) & 0xF0) | backlight_mode)

    def text(self, text, line, align='left'):
        self.write(LINES.get(line, LINES[1]))
        text, other_lines = self.get_text_line(text)
        text = getattr(text, ALIGN_FUNC.get(align, 'ljust'))(self.width)
        for char in text:
            self.write(ord(char), mode=1)
        if other_lines and line <= self.rows - 1:
            self.text(other_lines, line + 1, align=align)

    def backlight(self, turn_on=True):
        self.backlight_status = turn_on
        self.write(0)

    def get_text_line(self, text):
        line_break = self.width
        if len(text) > self.width:
            line_break = text[:self.width + 1].rfind(' ')
        if line_break < 0:
            line_break = self.width
        return text[:line_break], text[line_break:].strip()

    def clear(self):
        self.write(CLEAR_DISPLAY)

### App

In [ ]:
from time import sleep
import sqlite3
import datetime

DB_NAME = 'data.db'


class Mailbox:
    def __init__(self):
        with sqlite3.connect(DB_NAME) as con:
            cur = con.cursor()
            cur.execute('create table if not exists events (timestamp text, event_type text)')
            con.commit()

        self.mail_count = self._get_count()
        self._lcd = LCD()
        self._update_lcd()

    def receive_mail(self):
        self.mail_count += 1
        with sqlite3.connect(DB_NAME) as con:
            cur = con.cursor()
            cur.execute("insert into events values (datetime('now', 'localtime'), 'new_mail')")
            con.commit()
        self._update_lcd()

    def clear(self):
        self.mail_count = 0
        with sqlite3.connect(DB_NAME) as con:
            cur = con.cursor()
            cur.execute("insert into events values (datetime('now', 'localtime'), 'clean')")
            con.commit()
        self._update_lcd()
    
    def history(self):
        with sqlite3.connect(DB_NAME) as con:
            cur = con.cursor()
            cur.execute("select * from events where datetime(timestamp) > datetime('now', '-7 days')")
            return cur.fetchall()

    def _update_lcd(self):
        self._lcd.text('Mails:', 1, align='center')
        self._lcd.text(str(self.mail_count), 2, align='center')

    def _update_lcd(self):
        self._lcd.text('Mails:', 1, align='center')
        self._lcd.text(str(self.mail_count), 2, align='center')

    def _get_count(self):
        with sqlite3.connect(DB_NAME) as con:
            cur = con.cursor()
            cur.execute("select * from events where event_type == 'clean' order by timestamp desc")
            last_clean = cur.fetchone()
            if not last_clean:
                last_clean = (datetime.min.isoformat(), )
            
            cur.execute("select count(*) from events where datetime(timestamp) > datetime(?)", (last_clean[0] , ))
            return cur.fetchone()[0]


m = Mailbox()

In [16]:
import telebot


token = "5235740063:AAFe0mkLrcV20GrCM1cgrjZ2S3eBqioMPIY"
bot = telebot.TeleBot(token=token, parse_mode=None)


def tranlate(t):
    return 'Новое письмо'.ljust(26) if t == 'new_mail' else 'Вы почистили ящик'.ljust(20)


@bot.message_handler(commands=['history'])
def get_history(message):
    events = m.history()
    history = 'Пусто' if len(events) == 0 else '\n'.join(map(lambda t: f"{tranlate(t[1])}`{t[0]}`", events))
    bot.reply_to(message, f'Вот история за последние 7 дней:\n\n{history}', parse_mode="MarkdownV2")


@bot.message_handler(commands=['status'])
def get_status(message):
    bot.reply_to(message, f"Новых писем: {m.mail_count}")


@bot.message_handler(commands=['empty'])
def empty_mailbox(message):
    m.clear()
    bot.reply_to(message, "Ящик почищен")


@bot.message_handler(func=lambda _: True)
def unknown_command(message):
    bot.reply_to(message, "Извини, я не знаю такую команду")


bot.infinity_polling()

2022-04-13 22:14:26,525 (__init__.py:621 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2022-04-13 22:14:26,530 (__init__.py:623 MainThread) ERROR - TeleBot: "Break infinity polling"


In [ ]:
# подгатавливаем пины
GPIO.setmode(GPIO.BCM)
GPIO.setup(TRIG, GPIO.OUT)
GPIO.setup(ECHO, GPIO.IN)

# расчитываем расстояние пустого ящика
red_d = sum([get_distance() for _ in range(3)]) / 3
print(f'Референсное значение: {red_d}')

try:
    while True:
        d = get_distance()
        print(d, end='\r')
        if abs(red_d - d) > red_d/2:
            m.receive_mail()
finally:
    GPIO.cleanup()

In [17]:
GPIO.cleanup()

/tmp/ipykernel_2503/1609612480.py:1: RuntimeWarning: No channels have been set up yet - nothing to clean up!  Try cleaning up at the end of your program instead!
  GPIO.cleanup()
